## 优化问题

### SOFM保序特定验证

### 三角形区域

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import random
from math import sqrt
from numpy import linspace, zeros, array, amin, where
from PIL import Image
import os

class ImageSequenceGIF:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_frame_from_plt(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_temp_frames(self):
        for f in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, f))
        self.frames = []
        self.frame_count = 0

# 数据生成函数
def generate_data_triangle(num):
    pointdim = []
    for i in range(num):
        while True:
            x = random.uniform(0, 1)
            y = random.uniform(0, sqrt(3) / 2)
            y_limit = sqrt(3) * (0.5 - abs(x - 0.5))
            if y > y_limit:
                continue
            pointdim.append([x, y])
            break
    return np.array(pointdim)

# 可视化函数
def show_data(data, lineflag=0, title=''):
    if lineflag == 0:
        plt.scatter(data[:, 0], data[:, 1], s=10, c='blue')
    else:
        plt.scatter(data[:, 0], data[:, 1], s=35, c='red')
        plt.plot(data[:, 0], data[:, 1], 'y-', linewidth=1)

    board_x = linspace(0, 1, 200)
    board_y = [sqrt(3) * (0.5 - abs(x - 0.5)) for x in board_x]
    plt.plot(board_x, board_y, 'c--', linewidth=1)
    plt.plot(board_x, zeros(len(board_x)), 'c--', linewidth=1)

    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.axis([-0.05, 1.05, -0.05, .9])

    if len(title) > 0:
        plt.title(title)

    plt.grid(True)
    plt.tight_layout()

# Win-Take-All (WTA) 函数
def WTA2(x, w):
    dist = np.array([(x - ww).dot(x - ww) for ww in w])
    return list(where(dist == amin(dist)))[0][0]

# 一维邻域查找函数
def neighborid1(id, row, r):
    if r <= 0:
        return [id]
    iddim = []
    for i in range(-r, r + 1):
        if 0 <= id + i < row:
            iddim.append(id + i)
    return iddim

# 竞争更新函数
def compete1(x, w, eta, r):
    for xx in x:
        id = WTA2(xx, w)
        iddim = neighborid1(id, w.shape[0], r)
        for iidd in iddim:
            w[iidd] = w[iidd] + eta * (xx - w[iidd])
    return w

# 生成不同配置的 GIF
configurations = [
    {'SAMPLE_NUM': 100, 'NEURAL_NUM': 15, 'TRAIN_NUM': 200, 'gif_file': 'GIF/gif3-1.gif'},
    {'SAMPLE_NUM': 200, 'NEURAL_NUM': 50, 'TRAIN_NUM': 200, 'gif_file': 'GIF/gif3-2.gif'},
    {'SAMPLE_NUM': 300, 'NEURAL_NUM': 100, 'TRAIN_NUM': 200, 'gif_file': 'GIF/gif3-3.gif'},
]

for config in configurations:
    SAMPLE_NUM = config['SAMPLE_NUM']
    NEURAL_NUM = config['NEURAL_NUM']
    TRAIN_NUM = config['TRAIN_NUM']
    gif_file = config['gif_file']

    # 动态学习率和邻域半径
    ETA_BEGIN = 0.3
    ETA_END = 0.01
    RATIO_BEGIN = 10
    RATIO_END = 0

    # 数据生成和权重初始化
    x_data = generate_data_triangle(SAMPLE_NUM)
    W = np.random.rand(NEURAL_NUM, x_data.shape[1])
    W[:, 1] *= sqrt(3) / 2

    # 初始化 GIF 类
    gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file=gif_file)

    # 训练和可视化过程
    for i in range(TRAIN_NUM):
        eta = (ETA_BEGIN - ETA_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + ETA_END
        ratio = int((RATIO_BEGIN - RATIO_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + RATIO_END)

        W = compete1(x_data, W, eta, ratio)

        plt.clf()
        show_data(x_data, 0)
        show_data(W, 1, f"Step: {i+1}/{TRAIN_NUM}, R: {ratio}, eta: {eta:.2f}")
        plt.draw()
        gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

    # 保存 GIF
    gif_maker.save_gif(duration=100, last_frame_duration=500)
    gif_maker.clear_temp_frames()


GIF saved as GIF/gif3-1.gif
GIF saved as GIF/gif3-2.gif
GIF saved as GIF/gif3-3.gif


##### 训练参数

 周期：200；半径: $R: 10\to 0$; 学习率 $\eta:0.3\to0.01$  
 
----
##### 变量参数
样本数量：100；神经元数量：15  

![](GIF/gif3-1.gif)

样本数量：200; 神经元个数：50

![](GIF/gif3-2.gif)

样本数量：300; 神经元个数：100

![](GIF/gif3-3.gif)

#### "保序"性

自组织特征映射（SOFM）的“保序”特性指的是在网络的拓扑结构中，输入空间中的相邻数据点会映射到网络中的相邻神经元上，从而保留数据的空间拓扑结构。

* 相邻关系的保留：在SOFM训练过程中，输入空间中相似或距离较近的数据点往往会被映射到神经元网格中彼此相邻或接近的神经元上。
* 网络的自适应调整：OFM在训练过程中使用了竞争和协作的机制，获胜神经元及其邻域中的神经元会根据输入数据点进行权重更新。随着训练的进行，神经元权重不断调整，逐步贴合输入数据的分布，并将输入数据的相邻关系保留下来。
* 有序结构的形成：SOFM网络通过竞争和邻域更新，逐渐形成一个有序的、保序的映射结构。即使输入数据具有复杂的结构，例如多维空间的数据分布，SOFM仍然可以通过二维或一维网格结构将数据有序地映射到神经元网络中。

#### 矩形区域

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import amin, array, where, vstack
from PIL import Image
import os

class ImageSequenceGIF:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_frame_from_plt(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_temp_frames(self):
        for f in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, f))
        self.frames = []
        self.frame_count = 0

# 数据生成函数
def generate_data_rect(num):
    x_ = np.random.rand(num)
    y_ = np.random.rand(num)
    xy = vstack((x_, y_))
    return xy.T

# 显示数据点和方形网格神经元的函数
def show_data(data):
    plt.scatter(data[:, 0], data[:, 1], s=10, c='blue', label='Train Data')
    plt.vlines(0, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.vlines(1, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.hlines(0, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.hlines(1, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.axis([-0.1, 1.1, -0.1, 1.2])
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.grid(True)

def show_W(w):
    w_ = w.reshape(-1, 2)
    plt.scatter(w_[:, 0], w_[:, 1], s=40, c='red', label='Node')

    # 绘制连接线，形成方形网格结构
    for ww in w:
        plt.plot(ww[:, 0], ww[:, 1], color='red', linewidth=1, linestyle='--')
    for ww in w.transpose((1, 0, 2)):
        plt.plot(ww[:, 0], ww[:, 1], color='red', linewidth=1, linestyle='--')

    plt.legend(loc="upper right")
    plt.tight_layout()

# Win-Take-All (WTA) 函数
def WTA2(x, w):
    col = w.shape[1]
    dist = array([(x - ww).dot(x - ww) for ww in w.reshape(-1, 2)])
    id = list(where(dist == amin(dist)))[0][0]
    idx = id % col
    idy = id // col
    return (idx, idy)

# 二维邻域查找函数
def neighborid1(id, row, col, r):
    if r <= 0:
        return [id]

    iddim = []
    idx, idy = id

    for i in range(-r, r + 1):
        if idy + i < 0 or idy + i >= row:
            continue

        for j in range(-r, r + 1):
            if idx + j < 0 or idx + j >= col:
                continue
            iddim.append((idx + j, idy + i))

    return iddim

# 竞争更新函数
def compete1(x, w, eta, r):
    for xx in x:
        id = WTA2(xx, w)
        iddim = neighborid1(id, w.shape[0], w.shape[1], r)

        for iidd in iddim:
            idx, idy = iidd
            w[idy, idx] = w[idy, idx] + eta * (xx - w[idy, idx])

    return w

# 配置列表
configurations = [
    {'SAMPLE_NUM': 500, 'GRID_SIZE': (10, 10), 'TRAIN_NUM': 100, 'ETA_BEGIN': 0.1, 'ETA_END': 0.01, 'RATIO_BEGIN': 3, 'RATIO_END': 0, 'gif_file': 'GIF/gif4-1.gif'},
    {'SAMPLE_NUM': 200, 'GRID_SIZE': (10, 10), 'TRAIN_NUM': 100, 'ETA_BEGIN': 0.1, 'ETA_END': 0.01, 'RATIO_BEGIN': 3, 'RATIO_END': 0, 'gif_file': 'GIF/gif4-2.gif'},
    {'SAMPLE_NUM': 2000, 'GRID_SIZE': (10, 10), 'TRAIN_NUM': 100, 'ETA_BEGIN': 0.1, 'ETA_END': 0.01, 'RATIO_BEGIN': 3, 'RATIO_END': 0, 'gif_file': 'GIF/gif4-3.gif'},
    {'SAMPLE_NUM': 200, 'GRID_SIZE': (5, 5), 'TRAIN_NUM': 100, 'ETA_BEGIN': 0.1, 'ETA_END': 0.01, 'RATIO_BEGIN': 3, 'RATIO_END': 0, 'gif_file': 'GIF/gif4-4.gif'},
]

# 训练过程
for config in configurations:
    SAMPLE_NUM = config['SAMPLE_NUM']
    GRID_ROWS, GRID_COLS = config['GRID_SIZE']
    NEURAL_NUM = GRID_ROWS * GRID_COLS
    TRAIN_NUM = config['TRAIN_NUM']
    ETA_BEGIN = config['ETA_BEGIN']
    ETA_END = config['ETA_END']
    RATIO_BEGIN = config['RATIO_BEGIN']
    RATIO_END = config['RATIO_END']
    gif_file = config['gif_file']

    # 数据生成和权重初始化
    x_data = generate_data_rect(SAMPLE_NUM)
    W = np.stack((np.random.rand(GRID_ROWS, GRID_COLS), np.random.rand(GRID_ROWS, GRID_COLS)), axis=2)

    # 初始化 GIF 类
    gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file=gif_file)

    # 训练和可视化过程
    for i in range(TRAIN_NUM):
        eta = (ETA_BEGIN - ETA_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + ETA_END
        ratio = int((RATIO_BEGIN - RATIO_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + RATIO_END)

        W = compete1(x_data, W, eta, ratio)

        plt.clf()
        show_data(x_data)
        show_W(W)
        plt.title(f"Step: {i + 1}/{TRAIN_NUM}, R: {ratio}, eta: {eta:.2f}")
        plt.draw()
        gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

    # 保存 GIF
    gif_maker.save_gif(duration=100, last_frame_duration=500)
    gif_maker.clear_temp_frames()
plt.show()


GIF saved as GIF/gif4-1.gif
GIF saved as GIF/gif4-2.gif
GIF saved as GIF/gif4-3.gif
GIF saved as GIF/gif4-4.gif


##### 训练参数

 周期：100；半径: $R: 3\to 0$; 学习率 $\eta:0.1\to0.01$  
 
----
##### 变量参数
样本数量：500；神经元数量：10*10 

![](GIF/gif4-1.gif)

样本数量：200; 神经元个数：10*10

![](GIF\gif4-2.gif)

样本数量：2000; 神经元个数：10*10

![](GIF/gif4-3.gif)

样本数量：200; 神经元个数：5*5

![](GIF/gif4-4.gif)

### 旅行商问题

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import amin, array, where
from PIL import Image
import os
import random

class ImageSequenceGIF:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_frame_from_plt(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_temp_frames(self):
        for f in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, f))
        self.frames = []
        self.frame_count = 0

# 旅行商问题数据点
x_data = np.array([[236, 53], [408, 79], [909, 89], [115, 264], [396, 335], 
                   [185, 456], [699, 252], [963, 317], [922, 389], [649, 515]])
x_data = np.array([[xy[0] / 1000, xy[1] / 600] for xy in x_data])

# 显示数据点和路径
def show_data(data, lineflag=0, title=''):
    if lineflag == 0:
        plt.scatter(data[:, 0], data[:, 1], s=10, c='blue', label='View Site')
    else:
        plt.scatter(data[:, 0], data[:, 1], s=35, c='red', label='Neural Position')
        plt.plot(data[:, 0], data[:, 1], 'y--', linewidth=1)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.axis([-0.05, 1.05, -0.05, 1.05])
    if len(title) > 0:
        plt.title(title)
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

# Win-Take-All (WTA) 函数
def WTA2(x, w):
    dist = array([(x - ww).dot(x - ww) for ww in w])
    return list(where(dist == amin(dist)))[0][0]

# 循环邻域查找函数
def neighborid10(id, row, r):
    if r <= 0:
        return [id]

    iddim = []
    for i in range(-r, r + 1):
        iidd = id + i
        if iidd < 0:
            iidd += row
        if iidd >= row:
            iidd -= row

        if iidd not in range(0, row):
            continue
        iddim.append(iidd)

    return iddim

# 竞争更新函数，带有噪声
def compete1(x, w, eta, r):
    for xx in x:
        id = WTA2(xx, w)
        iddim = neighborid10(id, w.shape[0], r)

        for iidd in iddim:
            noise = (np.random.rand(2) - 0.5) * eta ** 2  # 随机噪声
            w[iidd] = w[iidd] + eta * (xx - w[iidd]) + noise

    return w

# 参数设置
SAMPLE_NUM = x_data.shape[0]
NEURAL_NUM = SAMPLE_NUM
TRAIN_NUM =  200  # 增加训练次数
ETA_BEGIN = 0.3
ETA_END = 0.01
RATIO_BEGIN = 2  # 增大初始半径
RATIO_END = 0

# 初始化权重矩阵
W = np.random.rand(NEURAL_NUM, x_data.shape[1])

# 初始化 GIF 类
gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file='GIF/tsp_solution.gif')

# 训练过程
for i in range(TRAIN_NUM):
    eta = (ETA_BEGIN - ETA_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + ETA_END
    ratio = int((RATIO_BEGIN - RATIO_END) * (TRAIN_NUM - i) / (TRAIN_NUM - 1) + RATIO_END)

    W = compete1(x_data, W, eta, ratio)

    plt.clf()
    show_data(x_data, 0)
    show_data(W, 1, f"Step: {i + 1}/{TRAIN_NUM}, R: {ratio}, eta: {eta:.2f}")
    plt.draw()
    gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

# 保存 GIF
gif_maker.save_gif(duration=100, last_frame_duration=500)
gif_maker.clear_temp_frames()
plt.show()


GIF saved as GIF/tsp_solution.gif


训练结果：反复调整了多次，尝试了循环+随机噪声的方式，以及包括学习率递减方式修改、初期添加随机扰动等方式，使其稳定在两个迷失节点+两个居中节点

![](GIF/tsp_solution.gif)